In [1]:
pdbs = ['7gqu', '7yg5', '8aq6', '8f4s', '8jp0', '8qni', '8uk6', '8v81', '9dnm']
upper_pdbs = [p.upper() for p in pdbs]

In [2]:
import os, sys

cur_dir = os.getcwd()

In [3]:
import importlib.util
from pathlib import Path

def import_function(module, function): # e.g. "prepare_fpocket_features" from "MEF-AlloSite/3-Prepare_fpocket_features.py"
    # Define a module name (cannot start with a number)
    module_name = module.split("-")[-1].replace(".py", "").lower() + "_module"
    
    # Load the module dynamically
    spec = importlib.util.spec_from_file_location(module_name, Path(module))
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)
    
    # Now import the function
    return getattr(module, function)

# Make predictions

- Adjusted one package in environments.yml
- Divided requirements.txt into pip and conda

<br>

## Place PDBs as expected after `1-Download_PDBs.py`

- Changed repo's `data` folder to `orig_data`

In [ ]:
for pdb in pdbs:
    os.system(f"cp ../structures/{pdb}.pdb MEF-AlloSite/data/pdbs/{pdb.upper()}.pdb")

## Find pockets as in `2-Find_pockets.py`

In [4]:
import sys
sys.path.append("MEF-AlloSite")

In [5]:
from uti import protein_uti

In [ ]:
protein_uti.run_fpocket(protein_list=upper_pdbs,
                        output_name="test",
                        chain_selection=False)

## `3-Prepare_fpocket_features`

- Added `if __name__ == "__main__":` to avoid execution when importing.
- Adjusted info retrieval from ASD dataset `protein_uti.get_info_from_ads(pdb)`; these PDBs are not going to be there.
- Adjusted pocket labelling that used ASD info; gave all label 0.

In [6]:
prepare_fpocket_features = import_function("MEF-AlloSite/3-Prepare_fpocket_features.py", "prepare_fpocket_features")

In [ ]:
prepare_fpocket_features(protein_list=upper_pdbs,
                input_path="MEF-AlloSite/data/test",
                output_file_name="test")

os.chdir(cur_dir)

## `4-Fix_pockets`

- Added `if __name__ == "__main__":` to avoid execution when importing.

In [7]:
fix_pocket = import_function("MEF-AlloSite/4-Fix_pockets.py", "fix_pocket")

In [ ]:
fix_pockets(protein_list=upper_pdbs,
                output_name="test")

os.chdir(cur_dir)

## `5-Prepare_Other_Features`

- Added `if __name__ == "__main__":` to avoid execution when importing.

In [8]:
prepare_all_features = import_function("MEF-AlloSite/5-Prepare_Other_Features.py", "prepare_all_features")

In [9]:
prepare_all_features(protein_list=upper_pdbs,
                     input_file_name="test",
                     fpocket_filename="test.csv")

os.chdir(cur_dir)

rm: cannot remove 'test.fasta': No such file or directory


protein 7GQU
sequnce LRVKEGIT
7GQU pocket21_atm_fixed.pdb
sequnce GNILQQQGEK
7GQU pocket2_atm_fixed.pdb
sequnce SRCQMGNCR
7GQU pocket9_atm_fixed.pdb
sequnce KPVKVMAYAAITCTGFDMEYYMKILHFEDKQQ
7GQU pocket14_atm_fixed.pdb
sequnce KAKKSRQISH
7GQU pocket16_atm_fixed.pdb
sequnce RPFFEGTRQQSSH
7GQU pocket22_atm_fixed.pdb
sequnce HSSFKPQATGYGKSFSPLISLMDQQYDEHGHDRADRPNYRYAFSRKHHIAFGMGINKAKSQERGRGQ
7GQU pocket23_atm_fixed.pdb
sequnce RQLSFEKVM
7GQU pocket4_atm_fixed.pdb
sequnce EWR
7GQU pocket19_atm_fixed.pdb
sequnce AEKHS
7GQU pocket1_atm_fixed.pdb
sequnce HEPSDEIQD
7GQU pocket15_atm_fixed.pdb
sequnce KNIP
7GQU pocket6_atm_fixed.pdb
sequnce YCPTIHGAPKEN
7GQU pocket11_atm_fixed.pdb
sequnce SSEKLYK
7GQU pocket13_atm_fixed.pdb
sequnce WSEDRK
7GQU pocket17_atm_fixed.pdb
sequnce YVNIPR
7GQU pocket20_atm_fixed.pdb
sequnce FLWPWKHSERQ
7GQU pocket7_atm_fixed.pdb
sequnce PNEVTK
7GQU pocket10_atm_fixed.pdb
sequnce YFHSLQQLSI
7GQU pocket5_atm_fixed.pdb
sequnce KPSKMQYGADI
7GQU pocket8_atm_fixed.pdb
sequnce

## `7-Make_predictions`

In [9]:
current_dir = f"{cur_dir}/MEF-AlloSite"
import pandas as pd

### Prepare individual .csv

In [19]:
# def split_training_set(filename,test_number):
filename="test_ready"

# training_protein_files = f"{current_dir}/data/test_one_by_one/test_{test_number}"
training_protein_files = f"{current_dir}/data/test_one_by_one/test"#_{test_number}" ##############################################
if not os.path.exists(training_protein_files):
    os.makedirs(training_protein_files)
os.chdir(current_dir)
data = pd.read_csv(f"{filename}.csv",low_memory=False)
# Split the 'protein_pocket' column by '_' and expand it into two separate columns
data[['protein_id', 'pocket_number']] = data['protein_pocket'].str.split('_', expand=True)

os.chdir(training_protein_files)
protein_list = []
for key, value in data.iterrows():
    protein_list.append(value["protein_id"])
protein_list = set(protein_list)

for target in protein_list:
    split_data = data[data["protein_id"] == str(target)]
    #shuffled_data = split_data.sample(frac=1)
    split_data.to_csv('%s.csv' % target, header=True)

In [20]:
os.chdir(cur_dir)

### Make predictions

In [4]:
from autogluon.tabular import TabularPredictor

def model_fn(model_dir):
    """loads model from previously saved artifact"""
    model = TabularPredictor.load(model_dir, require_py_version_match=False)  # or model = MultiModalPredictor.load(model_dir) for example
    model.persist_models()  # This line only works for TabularPredictor
    return model

/data/fnerin/envs_other_tools/mef-allosite/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# def validation_models(model_path, test_list, test_number,model_info, repeat_number=3, model_selection="MEF-AlloSite"):
model_path="MEF-AlloSite/AutogluonModels"
model_info="provided"
test_list=upper_pdbs
# test_number
repeat_number=3
model_selection="MEF-AlloSite"

In [29]:
import numpy as np

# fpocket_features = ["Label", 'Score', 'Druggability Score', 'Number of Alpha Spheres', "Total SASA",
#                     'Polar SASA', 'Apolar SASA',
#                     'Volume', 'Mean local hydrophobic density', 'Mean alpha sphere radius',
#                     'Mean alp. sph. solvent access',
#                     'Apolar alpha sphere proportion', 'Hydrophobicity score', 'Volume score', 'Polarity score',
#                     'Charge score',
#                     "Proportion of polar atoms",
#                     'Alpha sphere density', 'Cent. of mass - Alpha Sphere max dist', 'Flexibility', ]
gradient_boruta = ["Label", 'Druggability Score', 'Number of Alpha Spheres', 'Apolar SASA',
                   'biopython.csvcharge_at_pH', '_SecondaryStrD1100']
random_boruta = ["Label", 'Score', 'Druggability Score', 'Number of Alpha Spheres', 'Total SASA',
                 'Polar SASA', 'Apolar SASA', 'Volume', 'Mean local hydrophobic density',
                 'Apolar alpha sphere proportion', 'Hydrophobicity score', 'Charge score',
                 'Proportion of polar atoms', 'Alpha sphere density', 'Cent. of mass - Alpha Sphere max dist',
                 'biopython.csvmw', 'biopython.csvcharge_at_pH', 'QSOSW35', 'QSOSW37', 'QSOgrant37',
                 '_SecondaryStrC2', '_PolarityC1']
adaboosting = ["Label",'Score', 'Number of Alpha Spheres', 'Mean alpha sphere radius',
               'Druggability Score', 'biopython.csvaromaticty',
               'biopython.csvisoelectric_point']
gradient_boosting = ["Label",'Number of Alpha Spheres', 'biopython.csvmw', 'Druggability Score',
                     'QSOgrant34', 'Score', 'QSOgrant23', 'Mean local hydrophobic density',
                     'QSOgrant35', 'QSOSW17', 'QSOSW23', 'QSOgrant17', 'Volume', 'QSOSW35',
                     'GG', '_SecondaryStrD1075', 'QR', 'biopython.csvcharge_at_pH',
                     'Flexibility', 'biopython.csvaromaticty', 'Mean alpha sphere radius',
                     'biopython.csvinstability']

# average_f1_1 = []
# average_ave_pre = []
# average_roc_score = []
repeats = {pdb: None for pdb in test_list} #######################################################


for repeat in range(0,repeat_number):
    # passer = model_fn(f"{model_path}/{model_info}_{repeat}_passer")
    gradient_boruta_model = model_fn(f"{model_path}/{model_info}_{repeat}_gradient_boruta")
    random_boruta_model = model_fn(f"{model_path}/{model_info}_{repeat}_random_boruta")
    adaboosting_model = model_fn(f"{model_path}/{model_info}_{repeat}_adaboosting")
    gradient_boosting_model = model_fn(f"{model_path}/{model_info}_{repeat}_gradient_boosting")

    # average_pre_list = []
    # roc_score_list = []
    # f1_1_list = []

    for protein in test_list:
        # filename = f"{current_dir}/data/test_one_by_one/test_{test_number}/{protein}.csv"
        filename = f"{current_dir}/data/test_one_by_one/test/{protein}.csv" #######################################################
        data_test = pd.read_csv(filename, low_memory=False)
        # passer_test = data_test.loc[:, fpocket_features]
        gradient_boruta_test = data_test.loc[:, gradient_boruta]
        random_boruta_test = data_test.loc[:, random_boruta]
        adaboosting_test = data_test.loc[:, adaboosting]
        gradient_boosting_test = data_test.loc[:, gradient_boosting]

        # # find the location of 1 on y_true
        # y_true = data_test["Label"]

        # make prediction for unseen data
        # y_proba_0 = passer.predict_proba(passer_test)
        y_proba_1 = gradient_boruta_model.predict_proba(gradient_boruta_test)
        y_proba_2 = random_boruta_model.predict_proba(random_boruta_test)
        y_proba_3 = adaboosting_model.predict_proba(adaboosting_test)
        y_proba_4 = gradient_boosting_model.predict_proba(gradient_boosting_test)
        if model_selection == "MEF-AlloSite":
            mef_allo_site_proba = np.mean(
                [y_proba_1.iloc[:, 1], y_proba_2.iloc[:, 1], y_proba_3.iloc[:, 1], y_proba_4.iloc[:, 1]], axis=0)
            selected_model_proba=mef_allo_site_proba
        # elif model_selection == "PaSSer2.0":
        #     selected_model_proba=y_proba_0.iloc[:,1]
        ##############################################################################################################
        probs = pd.DataFrame(data_test[["protein_id", "pocket_number"]]).assign(**{str(repeat): selected_model_proba})
        if repeats[protein] is None:
            repeats[protein] = probs
        else:
            repeats[protein] = repeats[protein].merge(probs, on=["protein_id", "pocket_number"])
        ##############################################################################################################
        
    #     highest_1_indices = np.argsort(selected_model_proba)[-1:]

    #     # Create an array of zeros
    #     y_pred_1 = np.zeros(len(selected_model_proba))

    #     # Set the highest 3 probabilities to 1 
    #     y_pred_1[highest_1_indices] = 1

    #     # Here, only three metrics have been used to improve clearity
    #     aver_ = average_precision_score(y_true, selected_model_proba)
    #     roc_auc_score_ = roc_auc_score(y_true, selected_model_proba)
    #     f1_score_1 = f1_score(y_true=y_true, y_pred=y_pred_1)

    #     average_pre_list.append(aver_)
    #     roc_score_list.append(roc_auc_score_)
    #     f1_1_list.append(f1_score_1)

    # average_ave_pre.append(statistics.mean(average_pre_list))
    # average_roc_score.append(statistics.mean(roc_score_list))
    # average_f1_1.append(statistics.mean(f1_1_list))

# return statistics.mean(average_ave_pre),statistics.mean(average_roc_score),statistics.mean(average_f1_1)

Found 1 mismatches between original and current metadata:
/data/fnerin/envs_other_tools/mef-allosite/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator SimpleImputer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/data/fnerin/envs_other_tools/mef-allosite/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/data/fnerin/envs_other_tools/mef-allosite/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle e

In [30]:
repeats

{'7GQU':    protein_id  pocket_number         0         1         2
 0        7GQU             15  0.063723  0.082450  0.092161
 1        7GQU             21  0.087498  0.091109  0.118045
 2        7GQU             22  0.216535  0.230017  0.160590
 3        7GQU             10  0.047808  0.072774  0.070317
 4        7GQU             23  0.542877  0.535565  0.619775
 5        7GQU             16  0.305667  0.260981  0.267908
 6        7GQU              3  0.062360  0.088162  0.108413
 7        7GQU              2  0.050524  0.081991  0.084090
 8        7GQU              6  0.050209  0.069359  0.062801
 9        7GQU             12  0.049479  0.061808  0.063503
 10       7GQU             18  0.148227  0.180826  0.168378
 11       7GQU             20  0.057498  0.070987  0.069138
 12       7GQU              8  0.066796  0.097279  0.112999
 13       7GQU             17  0.054197  0.063947  0.065258
 14       7GQU              5  0.055184  0.065880  0.062478
 15       7GQU             13  0

# Process

In [38]:
from Bio import PDB

def process_pocket(f):
    s = PDB.PDBParser(QUIET=True).get_structure("protein", f)
    return pd.DataFrame(
        {
            "auth_asym_id": chain.id,
            "auth_seq_id": res.get_id()[1],
            "pdbx_PDB_ins_code": res.get_id()[2] if res.get_id()[2] != " " else "?"
        }
        for model in s
        for chain in model
        for res in chain
    ).drop_duplicates()

In [39]:
results = {}

for pdb, reps in repeats.items():
    pdb = pdb.lower()
    results[pdb] = {}

    
    selected_model_proba = reps.iloc[:,2:].mean(axis=1) # use average of the 3-repeats-average as final probabilities to transform to 1-labelled predictions with the same approach
    highest_1_indices = np.argsort(selected_model_proba)[-1:]

    # Create an array of zeros
    y_pred_1 = np.zeros(len(selected_model_proba))

    # Set the highest 3 probabilities to 1 
    y_pred_1[highest_1_indices] = 1

    for i, (ir, row) in enumerate(reps.iterrows()):
        pocket = f"pocket{row['pocket_number']}"
        results[pdb][pocket] = {
            "prob": selected_model_proba[i],
            "pred": int(y_pred_1[i]),
            "residues": process_pocket(f"MEF-AlloSite/data/fix_pockets/test/{pdb.upper()}_fixed/{pocket}_atm_fixed.pdb")
        }
        
results

{'7gqu': {'pocket15': {'prob': 0.07944463348637025,
   'pred': 0,
   'residues':   auth_asym_id  auth_seq_id pdbx_PDB_ins_code
   0            A          760                 ?
   1            A          762                 ?
   2            A          766                 ?
   3            A          790                 ?
   4            A          813                 ?
   5            A          814                 ?
   6            A          815                 ?
   7            A          816                 ?
   8            A          832                 ?},
  'pocket21': {'prob': 0.09888412648191054,
   'pred': 0,
   'residues':   auth_asym_id  auth_seq_id pdbx_PDB_ins_code
   0            A          561                 ?
   1            A          564                 ?
   2            A          588                 ?
   3            A          590                 ?
   4            A          657                 ?
   5            A          661                 ?
   6            A

In [14]:
import pickle

resultsf = "mef-allosite_results.pkl"

In [ ]:
with open(resultsf, "wb") as f:
    pickle.dump(results, f)

In [15]:
with open(resultsf, "rb") as f:
    results = pickle.load(f)

results

{'7gqu': {'pocket15': {'prob': 0.07944463348637025,
   'pred': 0,
   'residues':   auth_asym_id  auth_seq_id pdbx_PDB_ins_code
   0            A          760                 ?
   1            A          762                 ?
   2            A          766                 ?
   3            A          790                 ?
   4            A          813                 ?
   5            A          814                 ?
   6            A          815                 ?
   7            A          816                 ?
   8            A          832                 ?},
  'pocket21': {'prob': 0.09888412648191054,
   'pred': 0,
   'residues':   auth_asym_id  auth_seq_id pdbx_PDB_ins_code
   0            A          561                 ?
   1            A          564                 ?
   2            A          588                 ?
   3            A          590                 ?
   4            A          657                 ?
   5            A          661                 ?
   6            A

In [16]:
resultsd = {
    pdb: {
        pocket: {
            k: v if k != "residues" else v.astype(str).to_dict(orient="list")
            for k, v in pocketd.items()
        }
        for pocket, pocketd in pockets.items()
    } for pdb, pockets in results.items()
}

resultsd

{'7gqu': {'pocket15': {'prob': 0.07944463348637025,
   'pred': 0,
   'residues': {'auth_asym_id': ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A'],
    'auth_seq_id': ['760',
     '762',
     '766',
     '790',
     '813',
     '814',
     '815',
     '816',
     '832'],
    'pdbx_PDB_ins_code': ['?', '?', '?', '?', '?', '?', '?', '?', '?']}},
  'pocket21': {'prob': 0.09888412648191054,
   'pred': 0,
   'residues': {'auth_asym_id': ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A'],
    'auth_seq_id': ['561', '564', '588', '590', '657', '661', '662', '663'],
    'pdbx_PDB_ins_code': ['?', '?', '?', '?', '?', '?', '?', '?']}},
  'pocket22': {'prob': 0.20238072673479715,
   'pred': 0,
   'residues': {'auth_asym_id': ['A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A',
     'A'],
    'auth_seq_id': ['740',
     '752',
     '753',
     '763',
     '764',
     '765',
     '767',
     '834',
     '835',
     '861',
     '862',
     '863',
   

In [17]:
import json

In [18]:
with open(resultsf.replace(".pkl", ".json"), "w") as f:
    json.dump(resultsd, f)